In [1]:
import gym
import numpy as np
import copy, random, os, subprocess
from tensorflow.keras import layers, models

/opt/conda/lib/python3.7/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:23: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:24: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:25: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': pil_image.BICUBIC,
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:28: Depreca

In [2]:
MODEL_NAME = 'model04'
BUCKET = 'gs://etsuji-car-racing-v2'
os.environ['BUCKET'] = BUCKET
!gsutil mb -c regional -l us-west1 $BUCKET
!gsutil ls $BUCKET

Creating gs://etsuji-car-racing-v2/...
ServiceException: 409 A Cloud Storage bucket named 'etsuji-car-racing-v2' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [3]:
# Base model
class QValue:
    def __init__(self):
        self.model = self.build_model()

    def build_model(self):
        cnn_input = layers.Input(shape=(96, 96, 6), name='cnn_input')
        cnn1 = layers.Conv2D(32, (5, 5), padding='same',
                         use_bias=True, activation='relu',
                        name='cnn1')(cnn_input)
        pool1 = layers.MaxPooling2D((2, 2), name='pool1')(cnn1)
        cnn2 = layers.Conv2D(64, (5, 5), padding='same',
                         use_bias=True, activation='relu',
                        name='cnn2')(pool1)
        pool2 = layers.MaxPooling2D((2, 2), name='pool2')(cnn2)

        cnn_flatten = layers.Flatten(name='flatten')(pool2)
        action_input = layers.Input(shape=(5,), name='action_input')
        combined = layers.concatenate([cnn_flatten, action_input], name='concat')
        hidden1 = layers.Dense(1024, activation='relu', name='dense1')(combined)
        hidden2 = layers.Dense(512, activation='relu', name='dense2')(hidden1)
        q_value = layers.Dense(1, name='output')(hidden2)

        model = models.Model(inputs=[cnn_input, action_input], outputs=q_value)
        model.compile(loss='mse')
        return model

    def get_action(self, state):
        states = []
        actions = []
        for a in range(5):
            states.append(np.array(state))
            action_onehot = np.zeros(5)
            action_onehot[a] = 1
            actions.append(action_onehot)
  
        q_values = self.model.predict([np.array(states), np.array(actions)])
        optimal_action = np.argmax(q_values)
        return optimal_action, q_values[optimal_action][0]

In [4]:
def join_frames(o0, o1):
    return np.r_[o0.transpose(), o1.transpose()].transpose() 

In [5]:
def get_episode(environ, q_value, epsilon):
    episode = []
    o0 = environ.reset()
    o1 = copy.deepcopy(o0)
    total_r = 0

    if epsilon > 0:
        keep_count = 3
    else:
        keep_count = 1

    c = 0
    while True:
        if c % keep_count == 0: # Get new action
            if np.random.random() < epsilon:
                a = np.random.randint(5)
            else:
                a, _ = q_value.get_action(join_frames(o0, o1))
        c += 1
        o_new, r, done, inf = environ.step(a)                
        total_r += r

        # Terminate episode when total reward becomes negative
        if total_r < 0:
            done = 1

        if done:
            # Terminal state is to achive more than 990 or get out of the field.
            if total_r > 990 or r < -99:
                episode.append((join_frames(o0, o1), a, r, None))
            break
        else:
            episode.append((join_frames(o0, o1), a, r, join_frames(o1, o_new)))
        o0, o1 = o1, o_new

    print('epsilon={}, episode length={}, total rewards={}'.format(epsilon, len(episode), total_r))
    return episode, total_r

In [6]:
def train(environ, q_value, epsilon, checkpoint=0):
    if checkpoint > 0:
        filename = 'car-racing-v2-{}-{}.hd5'.format(checkpoint, MODEL_NAME)
        subprocess.run(['gsutil', 'cp', '{}/{}/{}'.format(BUCKET, MODEL_NAME, filename), './'])
        print('load model {}'.format(filename))
        q_value.model = models.load_model(filename)
        os.remove(filename)

    experience = []
    good_experience = []
    best_r = [-100, -100, -100]

    for n in range(checkpoint + 1, checkpoint + 1000):
        print('iteration {}'.format(n))

        total_len = 0
        if n % 3 == 0:
            print('Testing the current performance...')
            episode, total_r = get_episode(environ, q_value, epsilon=0)
            with open('result.txt', 'a') as f:
                f.write('{},{},{},{}\n'.format(n, epsilon, len(episode), total_r))
            filename = 'car-racing-v2-{}-{}.hd5'.format(n, MODEL_NAME)
            q_value.model.save(filename, save_format='h5')
            subprocess.run(['gsutil', '-m', 'cp',
                            '{}'.format(filename), '{}/{}/'.format(BUCKET, MODEL_NAME)])
            os.remove(filename)
            experience += episode
            total_len += len(episode)

        while total_len < 500:
            episode, total_r = get_episode(environ, q_value, epsilon)
            total_len += len(episode)
            experience += episode

            # Keep the top 3 episodes
            if total_r > min(best_r):
                best_r = best_r[1:] + [total_r]
                good_experience += episode
                if len(good_experience) > 999 * 3:
                    good_experience = good_experience[-999 * 3:]

            
        if len(experience) > 999 * 5: # remember last 5 episodes
            experience = experience[-999 * 5:]

        epsilon = (epsilon - 0.2) * 0.99 + 0.2

        print('Training the model...')
        # Use latest episode + past episodes (sampling) + top 3 episode (sampling)
        latest_experience = experience[-total_len:]
        past_experience = experience[:-total_len]
        examples = latest_experience + \
            random.sample(past_experience, min(len(past_experience), 999)) + \
            random.sample(good_experience, min(len(good_experience), 999))
        
        # Show some statistics
        print('experience length={}'.format(len(experience)))
        print('number of examples={}'.format(len(examples)))
        print('best total reward = ', best_r)
        np.random.shuffle(examples)
                        
        states, actions, labels = [], [], []
        for state, a, r, state_new in examples:
            states.append(np.array(state))

            action_onehot = np.zeros(5)
            action_onehot[a] = 1
            actions.append(action_onehot)
            
            if state_new is None:   # Terminal state
                q_new = 0
            else:
                _, q_new = q_value.get_action(state_new)
            labels.append(np.array(r + q_new))

        hist = q_value.model.fit(
            [np.array(states), np.array(actions)], np.array(labels),
            batch_size=50, epochs=10, verbose=0)
        print('loss = {}'.format(hist.history['loss']))

In [7]:
env = gym.make("CarRacing-v2", continuous=False)
q_value = QValue()
q_value.model.summary()

/opt/conda/lib/python3.7/site-packages/gym/core.py:330: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/opt/conda/lib/python3.7/site-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 cnn_input (InputLayer)         [(None, 96, 96, 6)]  0           []                               
                                                                                                  
 cnn1 (Conv2D)                  (None, 96, 96, 32)   4832        ['cnn_input[0][0]']              
                                                                                                  
 pool1 (MaxPooling2D)           (None, 48, 48, 32)   0           ['cnn1[0][0]']                   
                                                                                                  
 cnn2 (Conv2D)                  (None, 48, 48, 64)   51264       ['pool1[0][0]']                  
                                                                                              

2022-07-28 21:54:58.807432: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-28 21:54:58.817717: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-28 21:54:58.818350: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-28 21:54:58.819973: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [ ]:
train(env, q_value, epsilon=1.0, checkpoint=0)

iteration 1
epsilon=1.0, episode length=380, total rewards=-0.03771626297570238
epsilon=1.0, episode length=181, total rewards=-0.018181818181784964
Training the model...
experience length=561
number of examples=1122
best total reward =  [-100, -0.03771626297570238, -0.018181818181784964]


2022-07-28 21:55:06.848906: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200


loss = [3787211.25, 137.83840942382812, 45.38136291503906, 71.30708312988281, 31.55912208557129, 40.3258056640625, 36.6859245300293, 49.88913345336914, 52.35416793823242, 18.9506893157959]
iteration 2
epsilon=0.992, episode length=66, total rewards=-0.011036789297652144
epsilon=0.992, episode length=328, total rewards=-0.06417910447752884
epsilon=0.992, episode length=297, total rewards=-0.09702970297024893
Training the model...
experience length=1252
number of examples=1879
best total reward =  [-0.03771626297570238, -0.018181818181784964, -0.011036789297652144]
loss = [75.8316650390625, 19.269208908081055, 17.085594177246094, 14.495596885681152, 598.4967041015625, 4.036073684692383, 8.206777572631836, 8.700364112854004, 10.963042259216309, 8.247802734375]
iteration 3
Testing the current performance...
epsilon=0, episode length=65, total rewards=-0.06405228758169312


Copying file://car-racing-v2-3-model04.hd5 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

- [1/1 files][292.5 MiB/292.5 MiB] 100% Done                                    
Operation completed over 1 objects/292.5 MiB.                                    


epsilon=0.9840800000000001, episode length=395, total rewards=-0.03165467625888782
epsilon=0.9840800000000001, episode length=317, total rewards=-0.053968253968162516
Training the model...
experience length=2029
number of examples=2775
best total reward =  [-0.018181818181784964, -0.011036789297652144, -0.03165467625888782]
loss = [8.169157981872559, 6.145483016967773, 58.84730911254883, 5.335843086242676, 6.1102681159973145, 6.024691581726074, 592.198974609375, 2.950596332550049, 4.580855369567871, 5.049023628234863]
iteration 4
epsilon=0.9762392000000002, episode length=449, total rewards=-0.05617977528077067
epsilon=0.9762392000000002, episode length=107, total rewards=-0.08571428571427231
Training the model...
experience length=2585
number of examples=2554
best total reward =  [-0.018181818181784964, -0.011036789297652144, -0.03165467625888782]
loss = [4.861181735992432, 4.662554740905762, 4.838314056396484, 4.080642223358154, 5.404977321624756, 331.66192626953125, 103.64697265625,

...

Training the model...
experience length=4995
number of examples=2997
best total reward =  [62.1621621621666, 99.24812030075603, 125.19083969465893]
loss = [3.902280569076538, 3.2155704498291016, 3.276782274246216, 3.040729522705078, 3.0616016387939453, 2.8779335021972656, 2.390363931655884, 2.546079397201538, 2.622321605682373, 2.3133585453033447]
iteration 232
epsilon=0.27849101389094916, episode length=999, total rewards=30.00000000000103
Training the model...
experience length=4995
number of examples=2997
best total reward =  [62.1621621621666, 99.24812030075603, 125.19083969465893]
loss = [3.8431708812713623, 2.4763476848602295, 2.0009405612945557, 2.415804624557495, 1.9480758905410767, 1.7228760719299316, 2.6819217205047607, 1.7449816465377808, 1.60856032371521, 1.7321175336837769]
iteration 233
epsilon=0.27770610375203963, episode length=711, total rewards=-0.013559322034560867
Training the model...
experience length=4995
number of examples=2709
best total reward =  [62.162162162

Copying file://car-racing-v2-234-model04.hd5 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

| [1/1 files][292.5 MiB/292.5 MiB] 100% Done                                    
Operation completed over 1 objects/292.5 MiB.                                    


Training the model...
experience length=4995
number of examples=2997
best total reward =  [62.1621621621666, 99.24812030075603, 125.19083969465893]
loss = [2.6298418045043945, 2.180511474609375, 1.850675344467163, 2.4649035930633545, 2.2833921909332275, 1.8503175973892212, 1.6481455564498901, 1.682587742805481, 1.9595593214035034, 1.890679121017456]
iteration 235
epsilon=0.27615975228737405, episode length=864, total rewards=-0.033834586466548106
Training the model...
experience length=4995
number of examples=2862
best total reward =  [62.1621621621666, 99.24812030075603, 125.19083969465893]
loss = [2.4999847412109375, 2.081108331680298, 1.9440568685531616, 1.8365497589111328, 1.86399507522583, 1.7413723468780518, 1.584293007850647, 1.5902866125106812, 1.716611623764038, 1.6011890172958374]
iteration 236
epsilon=0.2753981547645003, episode length=555, total rewards=-0.044444444444900116
Training the model...
experience length=4995
number of examples=2553
best total reward =  [62.162162

Copying file://car-racing-v2-237-model04.hd5 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

| [1/1 files][292.5 MiB/292.5 MiB] 100% Done                                    
Operation completed over 1 objects/292.5 MiB.                                    


Training the model...
experience length=4995
number of examples=2666
best total reward =  [62.1621621621666, 99.24812030075603, 125.19083969465893]
loss = [2.549314022064209, 2.05466890335083, 2.2352616786956787, 1.7893273830413818, 1.81654691696167, 1.740776538848877, 1.9024478197097778, 1.529297113418579, 1.6377516984939575, 1.382238745689392]
iteration 238
epsilon=0.27389773148468677, episode length=499, total rewards=-3.350375532562566e-13
epsilon=0.27389773148468677, episode length=744, total rewards=-0.03191489361768271
Training the model...
experience length=4995
number of examples=3241
best total reward =  [62.1621621621666, 99.24812030075603, 125.19083969465893]
loss = [3.471451997756958, 2.9190688133239746, 2.3733010292053223, 2.4421370029449463, 2.0750138759613037, 2.0687544345855713, 2.144247531890869, 2.0393612384796143, 1.9641234874725342, 2.7135698795318604]
iteration 239
epsilon=0.2731587541698399, episode length=637, total rewards=-0.041610738255547036
Training the mod